In [24]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests
import asyncio
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [25]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def importrangeAllowAccessAutomation(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&includes_info_params=true&cros_files=false'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            data = response.json()
            print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

    

In [26]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def batchWriteRequest(service, spreadsheet_id, all_requests):
    # 요청들을 하나의 리스트로 병합
    # all_requests = []
    # for req in requests:
    #     all_requests.append(req)
    
    # 요청 바디 작성
    body = {
        'requests': all_requests
    }

    # batchUpdate 호출
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=body
    ).execute()
    
    return response


In [27]:
def col_to_index(col):
    index = 0
    for c in col:
        index = index * 26 + (ord(c) - ord('A') + 1)
    return index - 1

In [28]:
def convert_range_to_indices(sheet_id, cell_range):
    """
    주어진 셀 범위를 rowIndex와 columnIndex로 변환합니다.
    
    :param sheet_id: 변환할 시트의 ID
    :param cell_range: 변환할 셀 범위 (예: "A2:B")
    :return: 변환된 범위 딕셔너리
    """
    import re
    
    # A1 표기법을 정규 표현식으로 분리
    match = re.match(r"([A-Z]+)([0-9]+):([A-Z]+)([0-9]+)", cell_range)
    if not match:
        raise ValueError("셀 범위 형식이 올바르지 않습니다. 예: 'A2:B10'")
    
    start_col, start_row, end_col, end_row = match.groups()
    
    start_row_index = int(start_row) - 1
    end_row_index = int(end_row)
    start_col_index = col_to_index(start_col)
    end_col_index = col_to_index(end_col) + 1
    
    return {
        'sheetId': sheet_id,
        'startRowIndex': start_row_index,
        'endRowIndex':end_row_index,
        'startColumnIndex': start_col_index,
        'endColumnIndex': end_col_index
    }

In [29]:
def apply_protect_data(sheet_order_id, range_name):
    # 스프레드시트의 메타데이터에서 sheet_id 가져오기
    range_value = convert_range_to_indices(sheet_order_id, range_name);
    user_emails = ["daebong10x@gmail.com", "test-daebong@newdaebong.iam.gserviceaccount.com"]

        # 보호된 범위 설정
    request = {
        'addProtectedRange': {
            'protectedRange': {
                'range': range_value,
                'description': '수정을 원한다면 문의주세요',
                'editors': {
                    'users': user_emails,  # 이 셀을 편집할 수 있는 사용자
                }
            }
        }
    }

    return request

In [30]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_text(text, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    
    range_value = convert_range_to_indices(sheet_order_id, range_name);

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": range_value,
        }
    }
    return request

In [31]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_to_function(formula, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    # spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    range_value = convert_range_to_indices(sheet_order_id, range_name);

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {
                            "formulaValue": formula
                        }}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": range_value,
        }
    }
    return request

In [32]:
def apply_dropdown_validation(dropdown_range, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    # spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    range_value = convert_range_to_indices(sheet_order_id, range_name);
    
    # 드롭다운 유효성 검사 설정
    request = {
                'repeatCell': {
                    'range': range_value,
                    'cell': {
                        'dataValidation': {
                            'condition': {
                                'type': 'ONE_OF_RANGE',
                                'values': [
                                    {
                                        'userEnteredValue': f"={dropdown_range}"
                                    }
                                ]
                            },
                            'showCustomUi': True, # 드롭다운 화살표 표시
                            'strict': False # 경고 표시
                        }
                    },
                    'fields': 'dataValidation'
                }
    }
    
    return request

In [33]:
def apply_rename_request(sheet_id, title):
    request = {
                "updateSheetProperties": {
                    "properties": {
                        "sheetId": sheet_id,
                        "title": title,
                    },
                    "fields": "title"
                }
    }
    return request

In [34]:
def apply_index_request(sheet_id, index):
    request =  {
                "updateSheetProperties": {
                    "properties": {
                        "sheetId": sheet_id,
                        "index": index
                    },
                    "fields": "index"
                }
    }
    return request

In [35]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_name_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF($A$8="매핑모드",IFERROR(INDEX(\'상품목록\'!C:C, MATCH(1, (E{i}=\'상품목록\'!A:A) * (F{i}=\'상품목록\'!B:B), 0)), ""),    IF($A$8="일반모드",  D{i}, "" ))'
        requests.append(
            {
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":6,  # Column G (0-based index)
                    "endColumnIndex": 7
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        }
        )
    return requests

In [36]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_mapping_check_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF($A$8="매핑모드", IF(E{i}<>"", IF(G{i}<>"", "정상매핑", "매핑오류"), ""), "")'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":7,  # Column H (0-based index)
                    "endColumnIndex": 8
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [37]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_price_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=\'상품목록\'!BA{i}'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":9,  # Column J (0-based index)
                    "endColumnIndex": 10
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [38]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_order_status_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF($A$8="매핑모드",IF(ISNUMBER(SEARCH("정상매핑", H{i})),IF(AND(A{i}<>"", B{i}<>"", C{i}<>"", J{i}<>""), "정상입력", "필수입력사항 입력필요"),IF(H{i}="매핑오류", "매핑 확인필요", IF(OR(A{i}<>"", B{i}<>"", C{i}<>"", J{i}<>""), "매핑 확인필요", ""))),IF($A$8="일반모드",IF(AND(A{i}="", B{i}="", C{i}="", D{i}="", J{i}=""), "", IF(OR(A{i}="", B{i}="", C{i}="", D{i}="", J{i}=""), "필수입력사항 입력필요", "정상입력")),""))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":10,  # Column K (0-based index)
                    "endColumnIndex": 11
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [39]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_number_to_function_in_everyrow(sheet_order_id):

    requests = []

    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(IF(LEN(G{i})=0,NA(), 1))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": 11,  # Column J (0-based index)
                    "endColumnIndex": 12
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [40]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_code_number_to_function_in_everyrow(sheet_order_id, new_daebong_spreadsheet_title):

    requests = []

    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", {new_daebong_spreadsheet_title})'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": 19,  # Column T (0-based index)
                    "endColumnIndex": 20
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [41]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_to_function_in_everyrow(sheet_order_id, new_daebong_spreadsheet_title):

    requests = []

    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", {new_daebong_spreadsheet_title})'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": 19,  # Column J (0-based index)
                    "endColumnIndex": 20
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [42]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_delivery_company_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(VLOOKUP(G{i}, \'상품목록\'!K$6:M, 3, FALSE), "")'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":9,  # Column K (0-based index)
                    "endColumnIndex": 10
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [43]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_suppy_price_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(IF(INDEX(\'상품목록\'!L:L, MATCH(\'발주서\'!G{i}, \'상품목록\'!K:K, 0))*\'발주서\'!L{i}=0, "", INDEX(\'상품목록\'!L:L, MATCH(\'발주서\'!G{i}, \'상품목록\'!K:K, 0))*\'발주서\'!L{i}))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":52,  # Column J (0-based index)
                    "endColumnIndex": 53
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [44]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def find_last_row_in_spreadsheet(service, spreadsheet_id, column):
    """
    지정된 열에서 데이터가 있는 마지막 행의 인덱스를 반환합니다.
    """
    range_name = f'{column}:{column}'  # 예: 'D:D'
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=range_name
    ).execute()
    values = result.get('values', [])

    if not values:
        return 1  # 데이터가 없으면 첫 번째 행 반환
    else:
        return len(values) + 1  # 데이터가 있는 마지막 행의 다음 행 반환



def append_to_sheet_in_a_row(service, many_ids, sheet_order_id):
    last_row = find_last_row_in_spreadsheet(service, new_spreadsheet_id, 'D')
    # D열의 마지막 행 찾기
    # last_row_D = find_last_row(service, spreadsheet_id, 'D')
    results = []
    count = 0

    for order_sheet_title, new_spreadsheet_id in many_ids:
        this_row = last_row + count
        count += 1

        range_name_D = f'시트1!D{this_row}'
        request_D = apply_formula_to_text(order_sheet_title, range_name_D, sheet_order_id)

        # E열의 마지막 행 찾
        range_name_E = f'시트1!E{this_row}'
        request_E = apply_formula_to_text(new_spreadsheet_id, range_name_E, sheet_order_id)

        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{new_spreadsheet_id}"

        range_name_F = f'시트1!F{this_row}'
        request_F = apply_formula_to_text(spreadsheet_url, range_name_F, sheet_order_id)

        results.extend([request_D, request_E, request_F])

    return results 

    # return [request_D, request_E. request_F]



            


In [45]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def importrangeAllowAccessAutomation(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&token={creds.token}&includes_info_params=true&cros_files=false'
    # https://docs.google.com/spreadsheets/d/1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw/externaldata/addimportrangepermissions?donorDocId=1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4&token=AC4w5VjkIUgDXtWZH968yYaAfiyhZJcApg%3A1718330258244&includes_info_params=true&cros_files=false

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            # 응답 상태 코드 확인
            if response.status_code == 200:
                # JSON 응답 파싱
                data = response.json()
                # 파싱된 데이터 출력
                print(data)
                print(response);
            # data = response.json()
            # print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

    

In [52]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def copy_sheet_to_another_spreadsheet(service, source_spreadsheet_id, source_sheet_id, destination_spreadsheet_id):
    # Copy the sheet from the source spreadsheet to the destination spreadsheet
    request_body = {
        'destinationSpreadsheetId': destination_spreadsheet_id
    }
    
    response = service.spreadsheets().sheets().copyTo(
        spreadsheetId=source_spreadsheet_id,
        sheetId=source_sheet_id,
        body=request_body
    ).execute()
    
    return response['sheetId']

@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def main_copy_sheet(service_sheets, source_spreadsheet_id, destination_spreadsheet_id):

    #특정시트에 복사하기 
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=source_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_매핑모드 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_입금내역 = None
    sheet_order_id_상품목록 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "발주서매핑모드":
            sheet_order_id_매핑모드 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "누적발주":
            sheet_order_id_누적발주 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "입금내역":
            sheet_order_id_입금내역 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')

    new_sheet_order_id_발주서 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_발주서, destination_spreadsheet_id)
    new_sheet_order_id_매핑모드 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_매핑모드, destination_spreadsheet_id)
    new_sheet_order_id_누적발주 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_누적발주, destination_spreadsheet_id)
    new_sheet_order_id_입금내역 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_입금내역, destination_spreadsheet_id)
    new_sheet_order_id_상품목록 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_상품목록, destination_spreadsheet_id)


    return [new_sheet_order_id_발주서,new_sheet_order_id_누적발주,new_sheet_order_id_입금내역,new_sheet_order_id_상품목록, new_sheet_order_id_매핑모드]

@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def delete_sheet_by_title(service_sheets, destination_spreadsheet_id):
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=destination_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_입금내역 = None
    sheet_order_id_상품목록 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서" or sheet.get('properties', {}).get('title', '') == "발주서의 사본" :
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "누적발주" or sheet.get('properties', {}).get('title', '') == "누적발주의 사본":
            sheet_order_id_누적발주 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "입금내역" or sheet.get('properties', {}).get('title', '') == "입금내역의 사본":
            sheet_order_id_입금내역 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록" or sheet.get('properties', {}).get('title', '') == "상품목록의 사본":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')


    requests = []
    
    if sheet_order_id_상품목록:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_상품목록
            }
        })

    if sheet_order_id_입금내역:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_입금내역
            }
        })
    
    if sheet_order_id_발주서:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_발주서
            }
        })
    if sheet_order_id_누적발주:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_누적발주
            }
        })

    delete_request_body = {
        "requests": requests
    }

    if len(requests)>0:
        service_sheets.spreadsheets().batchUpdate(
            spreadsheetId=destination_spreadsheet_id,
            body=delete_request_body
        ).execute()


async def edit_sheet_title_by_title(service_sheets, destination_spreadsheet_id):
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=destination_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_입금내역 = None
    sheet_order_id_상품목록 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "누적발주":
            sheet_order_id_누적발주 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "입금내역":
            sheet_order_id_입금내역 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')

    requests = []
    # apply_rename_request(sheet_order_id_발주서, "발주서"),
    # apply_rename_request(sheet_order_id_누적발주, "누적발주"),
    # apply_rename_request(sheet_order_id_입금내역, "입금내역"),
    # apply_rename_request(sheet_order_id_상품목록, "상품목록"),
    
    
    if sheet_order_id_상품목록:
        requests.append(apply_rename_request(sheet_order_id_상품목록, "상품목록이전버전"))
    
    if sheet_order_id_발주서:
        requests.append(apply_rename_request(sheet_order_id_발주서, "발주서이전버전"))
        # requests.append({
        #     "deleteSheet": {
        #         "sheetId": sheet_order_id_발주서
        #     }
        # })
    if sheet_order_id_누적발주:
        requests.append(apply_rename_request(sheet_order_id_누적발주, "누적발주이전버전"))

    delete_request_body = {
        "requests": requests
    }

    if len(requests)>0:
        service_sheets.spreadsheets().batchUpdate(
            spreadsheetId=destination_spreadsheet_id,
            body=delete_request_body
        ).execute()


async def apply_formula_and_text(sheet_id_list, ordersheet_number_title, depositer_name):
    sheet_order_id_발주서 = sheet_id_list[0]
    sheet_order_id_누적발주 = sheet_id_list[1]
    sheet_order_id_입금내역 = sheet_id_list[2]
    sheet_order_id_상품목록 = sheet_id_list[3]
    sheet_order_id_매핑모드 = sheet_id_list[4]

    request_name_index = [
        apply_rename_request(sheet_order_id_발주서, "발주서"),
        apply_rename_request(sheet_order_id_누적발주, "누적발주"),
        apply_rename_request(sheet_order_id_입금내역, "입금내역"),
        apply_rename_request(sheet_order_id_상품목록, "상품목록"),
        apply_rename_request(sheet_order_id_매핑모드, "발주서매핑모드"),
        apply_index_request(sheet_order_id_발주서, 0),
        apply_index_request(sheet_order_id_누적발주, 1),
        apply_index_request(sheet_order_id_상품목록, 2),
        apply_index_request(sheet_order_id_입금내역, 3),
        apply_index_request(sheet_order_id_매핑모드, 4)
    ]
    requests = []

    ordersheet_number = int(ordersheet_number_title)

    result_입금자명_발주서 = apply_formula_to_text(str(depositer_name), "J4:J4", sheet_order_id_발주서)

    result_발주서번호_발주서 = apply_formula_to_text(str(ordersheet_number_title), "T4:T4", sheet_order_id_발주서)

    formula_예치금잔액_발주서 = f'=IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=516836762", "\'셀러발주서정보\'!D{ordersheet_number + 1}:D{ordersheet_number + 1}")'
    result_예치금잔액_발주서 = apply_formula_to_function(formula_예치금잔액_발주서, "K4:K4", sheet_order_id_발주서)

    formula_입금내역 = f'=IFERROR(QUERY(IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=322712525", "전체입금!A2:E"), "SELECT Col1, Col2, Col3, Col5 WHERE Col5 = {ordersheet_number_title} ORDER BY Col3 DESC"), "")'
    result_입금내역_입금내역 = apply_formula_to_function(formula_입금내역, "A2:A2", sheet_order_id_입금내역)

    formula_상품목록 = f'=IMPORTRANGE("1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4","\'시트1\'!A2:M930")'
    result_상품목록_상품목록= apply_formula_to_function(formula_상품목록, "AA1:AA1", sheet_order_id_상품목록)


    # formula_입금확인중_발주서 = f'=IFERROR(SUMIF(\'누적발주\'!H4:H, "입금확인중", \'누적발주\'!R4:R), 0)'
    # formula_추가입금필요_발주서 = f'=IFERROR(SUMIF(\'누적발주\'!H4:H, "입금확인필요", \'누적발주\'!R4:R), 0)'

    # result_입금확인중_발주서 = apply_formula_to_function(formula_입금확인중_발주서, "O4:O4", sheet_order_id_발주서)
    # result_추가입금필요_발주서 = apply_formula_to_function(formula_추가입금필요_발주서, "P4:P4", sheet_order_id_발주서)
    
    
    # result_주문상품_발주서 = apply_formula_product_name_to_function_in_everyrow(sheet_order_id_발주서)
    # result_상품매핑확인_발주서 = apply_formula_mapping_check_to_function_in_everyrow(sheet_order_id_발주서)
    # result_공급금액_발주서 = apply_formula_product_price_to_function_in_everyrow(sheet_order_id_발주서)
    # result_발주상태_발주서 = apply_formula_order_status_to_function_in_everyrow(sheet_order_id_발주서)
    # result_수량_발주서 = apply_formula_product_number_to_function_in_everyrow(sheet_order_id_발주서)
    # result_코드번호_발주서 = apply_formula_code_number_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)
    result_코드번호_발주서 = apply_formula_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)


    # requests.extend(result_주문상품_발주서)
    # requests.extend(result_상품매핑확인_발주서)
    # requests.extend(result_공급금액_발주서)
    # requests.extend(result_발주상태_발주서)
    # requests.extend(result_수량_발주서)
    # requests.extend(result_코드번호_발주서)
    print("result_코드번호_발주서")
    print(result_코드번호_발주서)
    

    # result_code_number_코드번호_발주서 = apply_formula_code_number_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)

    # result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D940", sheet_order_id_발주서)

    # requests.extend(result_dropdown_발주서)

    request_protect = [
        apply_protect_data(sheet_order_id_발주서, "J4:O4"),
        # apply_protect_data(sheet_order_id_발주서, "T1:Z940"),
        apply_protect_data(sheet_order_id_상품목록, "I1:BA1000"),
        # apply_protect_data(sheet_order_id_입금내역, "A1:Z940")
        ]

    request_function_index = [
        result_입금자명_발주서, result_예치금잔액_발주서, result_입금내역_입금내역, result_상품목록_상품목록, result_발주서번호_발주서
    ]   

    requests.extend(result_코드번호_발주서)
    requests.extend(request_function_index)
    requests.extend(request_name_index)
    requests.extend(request_protect)
    
    # return requests
    return requests



async def apply_formula_and_text_onemore(sheet_id_list, ordersheet_number_title):
    sheet_order_id_발주서 = sheet_id_list[0]
    sheet_order_id_누적발주 = sheet_id_list[1]
    sheet_order_id_입금내역 = sheet_id_list[2]
    sheet_order_id_상품목록 = sheet_id_list[3]

    requests = []

    ordersheet_number = int(ordersheet_number_title)

    # result_입금자명_발주서 = apply_formula_to_text(str(ordersheet_number_title), "J4:J4", sheet_order_id_발주서)

    # formula_예치금잔액_발주서 = f'=IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=516836762", "\'셀러발주서정보\'!D{ordersheet_number + 1}:D{ordersheet_number + 1}")'
    # result_예치금잔액_발주서 = apply_formula_to_function(formula_예치금잔액_발주서, "K4:K4", sheet_order_id_발주서)


    # formula_입금확인중_발주서 = f'=IFERROR(SUMIF(\'누적발주\'!H4:H, "입금확인중", \'누적발주\'!R4:R), 0)'
    # formula_추가입금필요_발주서 = f'=IFERROR(SUMIF(\'누적발주\'!H4:H, "입금확인필요", \'누적발주\'!R4:R), 0)'

    # result_입금확인중_발주서 = apply_formula_to_function(formula_입금확인중_발주서, "O4:O4", sheet_order_id_발주서)
    # result_추가입금필요_발주서 = apply_formula_to_function(formula_추가입금필요_발주서, "P4:P4", sheet_order_id_발주서)
    
    # requests.extend([result_입금확인중_발주서, result_추가입금필요_발주서])
    
    result_주문상품_발주서 = apply_formula_product_name_to_function_in_everyrow(sheet_order_id_발주서)
    result_상품매핑확인_발주서 = apply_formula_mapping_check_to_function_in_everyrow(sheet_order_id_발주서)
    result_공급금액_발주서 = apply_formula_product_price_to_function_in_everyrow(sheet_order_id_발주서)
    result_발주상태_발주서 = apply_formula_order_status_to_function_in_everyrow(sheet_order_id_발주서)
    result_수량_발주서 = apply_formula_product_number_to_function_in_everyrow(sheet_order_id_발주서)
    result_코드번호_발주서 = apply_formula_code_number_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)
    result_공급금액_상품목록 = apply_formula_suppy_price_to_function_in_everyrow(sheet_order_id_상품목록)

    requests.extend(result_주문상품_발주서)
    requests.extend(result_상품매핑확인_발주서)
    requests.extend(result_공급금액_발주서)
    requests.extend(result_발주상태_발주서)
    requests.extend(result_수량_발주서)
    requests.extend(result_코드번호_발주서)
    requests.extend(result_공급금액_상품목록)

    result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_발주서)
    # print(result_dropdown_발주서)

    requests.extend([result_dropdown_발주서])

    # formula_입금내역 = f'=IFERROR(QUERY(IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=322712525", "전체입금!A2:E"), "SELECT Col1, Col2, Col3, Col5 WHERE Col5 = {ordersheet_number_title} ORDER BY Col3 DESC"), "")'
    # result_입금내역_입금내역 = apply_formula_to_function(formula_입금내역, "A2:A2", sheet_order_id_입금내역)

    # formula_상품목록 = f'=IMPORTRANGE("1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4","\'시트1\'!A2:M999")'
    # result_상품목록_상품목록= apply_formula_to_function(formula_상품목록, "AA1:AA1", sheet_order_id_상품목록)
    # request_protect = [
    #     apply_protect_data(sheet_order_id_발주서, "J4:O4"),
    #     # apply_protect_data(sheet_order_id_발주서, "T1:Z940"),
    #     apply_protect_data(sheet_order_id_상품목록, "I1:BA1000"),
    #     # apply_protect_data(sheet_order_id_입금내역, "A1:Z940")
    #     ]


    # requests.extend([result_입금자명_발주서, result_예치금잔액_발주서, result_입금내역_입금내역, result_상품목록_상품목록, result_입금확인중_발주서, result_추가입금필요_발주서
                    #  ,result_dropdown_발주서])
    # requests.extend(result_code_number_코드번호_발주서)
    # requests.extend(request_name_index)
    # requests.extend(request_protect)
    
    return requests

    # result_입금내역_입금내역 = apply_formula_to_function(service_sheets, spreadsheet_id, sheet_title, formula, cell_range, sheet_order_id_발주서)

async def main(service_sheets, destination_spreadsheet_id, source_spreadsheet_id, spreadsheet_title, depositer_name):
    # await delete_sheet_by_title(service_sheets, destination_spreadsheet_id)
    await edit_sheet_title_by_title(service_sheets, destination_spreadsheet_id)
    sheet_id_list = await main_copy_sheet(service_sheets, source_spreadsheet_id, destination_spreadsheet_id)
    # sheet_id_list = [424543879, 762666408, 41723993, 1523662970]
    # sheet_id_list = [951548530, 1105113483, 693385300, 299949341]
    print(sheet_id_list)
    requests = await apply_formula_and_text(sheet_id_list, spreadsheet_title, depositer_name)
    print(requests)
    await batchWriteRequest(service_sheets, destination_spreadsheet_id, requests)
    # onemore_requests = await apply_formula_and_text_onemore(sheet_id_list, spreadsheet_title)
    # print(onemore_requests)
    # await batchWriteRequest(service_sheets, destination_spreadsheet_id, onemore_requests)


@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values
    


def filter_and_map(input_list):
    result = []
    for sublist in input_list:
        if len(sublist) > 1 and sublist[2]:  # 두 번째 요소가 비어있지 않은지 확인
            result.append([sublist[0], sublist[2]])
    return result

import time

if __name__ == "__main__":
    # destination_spreadsheet_id = "1G1qctmwr3pMKrhMHv1hMbvXrmmCI1TsJ5OLqKKg7U8c"
    source_spreadsheet_id = "1AdHR51GZnnyGuLs25c5Al3bgWX4TN9EXi2KgKBrOesY"
    archive_spreadsheet_id = "1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc"
    # product_spreadsheet_id = "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY" #000~100: 1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY
    # product_spreadsheet_id = "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY" #000~100: 1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY
    product_spreadsheet_id = "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY" # 411~510:1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo
    # product_spreadsheet_id = "1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4" #511 ~ 610
    
    # 211~310:1XDBIK9o2k4AVMcW2gfieDZ4qSBxcTW8npw596AY6KCY
    # 311~410:1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo
    # 411~510:1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo
    # 511~610:1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4

    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    user_data_list = get_values_from_range(service_sheets, archive_spreadsheet_id, "셀러발주서정보!A581:D581") #회원가입 연동 테이블

    # filtered_user_data = filter_and_map(user_data_list)
    filtered_user_data = [
# [581,	"1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw"]
# [581, "1-vrWkYDHqDmhtkV5ZylS_AvmbKjrfAncu1atMKQQ8f0"]
# [461, "104ZWc3RoB0WjkITYQLyFBLvkZ5UywvIJZHoWIha9iTk"]
# ["516",	"1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM", "485정준희"]
# ["485",	"1vVGjqAgaOFovhHhrcpGkHmGktJug0f0zKgYIERT2FTA", "485정준희"] 싸우고있어서 아직못함
# ["442",	"1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE", "임경섭(프루딧)"],
# ["555",	"1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw", "(주)마니쿡스"],
# ["523",	"1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg", "김지영"],
# ["339",	"1GCUBnyUr25uhB3HozLbiUQ5JngLhr7rfoNNR1cxAo58", "339슈거팜"]
["335",	"13bZWoE-oHVuPaPQXRYxIjDDAqmpvnqETYuP5S94Re08", "홀핸드"],
# ,["325",	"14SbtLIs4ZPfiaJh-kRb-TE4RtOcO-gw4knuy3MolLX8", "이호연(컨프래"]
# ,["114",	"1mZVlL4QwI2eHByQEpYs0dLYUQX4OcSa4tsdTJPR2VRI", "114"]
    ]
    # result = [["560", "1jhYbEM98arK-Xj5cSQ01P2Oa3kfgO6lNODw6tDqEFPU"], ["561", "1yLURni5N03TeMxF4RugKa9UAxJ6VyTstGztM2byv9sY"], ["562", "1jYeZFz3CNgXkBV3z1CsoD8k_e6xH-vZa3M0KboBD8rk"]]
    # print(filtered_user_data)
    # importrangeAllowAccessAutomation("1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw", product_spreadsheet_id, creds)
    for data in filtered_user_data:
        spreadsheet_title = data[0]
        destination_spreadsheet_id = data[1]
        print("spreadsheet_title", spreadsheet_title)
        depositer_name = ""
        if len(data) > 2:
            depositer_name = data[2]

        # if int(spreadsheet_title) < 580:
        #     print("580이하입니다")
        #     break
        await main(service_sheets, destination_spreadsheet_id, source_spreadsheet_id, spreadsheet_title, depositer_name)
        
       

        time.sleep(0.5)

        print("Calling importrangeAllowAccessAutomation for archive_spreadsheet_id")
        try:
            importrangeAllowAccessAutomation(destination_spreadsheet_id, archive_spreadsheet_id, creds)
            print("importrangeAllowAccessAutomation for archive_spreadsheet_id executed successfully")
        except Exception as e:
            print(f"Error during importrangeAllowAccessAutomation for archive_spreadsheet_id: {e}")


        print("Calling importrangeAllowAccessAutomation for product_spreadsheet_id")
        try:
            importrangeAllowAccessAutomation(destination_spreadsheet_id, product_spreadsheet_id, creds)
            print("importrangeAllowAccessAutomation for product_spreadsheet_id executed successfully")
        except Exception as e:
            print(f"Error during importrangeAllowAccessAutomation for product_spreadsheet_id: {e}")
        

spreadsheet_title 335
[112758672, 921632066, 24107480, 505352963, 580636407]
result_코드번호_발주서
[{'updateCells': {'range': {'sheetId': 112758672, 'startRowIndex': 11, 'endRowIndex': 12, 'startColumnIndex': 19, 'endColumnIndex': 20}, 'rows': [{'values': [{'userEnteredValue': {'formulaValue': '=IF(LEN(A12)=0,"", 335)'}}]}], 'fields': 'userEnteredValue'}}, {'updateCells': {'range': {'sheetId': 112758672, 'startRowIndex': 12, 'endRowIndex': 13, 'startColumnIndex': 19, 'endColumnIndex': 20}, 'rows': [{'values': [{'userEnteredValue': {'formulaValue': '=IF(LEN(A13)=0,"", 335)'}}]}], 'fields': 'userEnteredValue'}}, {'updateCells': {'range': {'sheetId': 112758672, 'startRowIndex': 13, 'endRowIndex': 14, 'startColumnIndex': 19, 'endColumnIndex': 20}, 'rows': [{'values': [{'userEnteredValue': {'formulaValue': '=IF(LEN(A14)=0,"", 335)'}}]}], 'fields': 'userEnteredValue'}}, {'updateCells': {'range': {'sheetId': 112758672, 'startRowIndex': 14, 'endRowIndex': 15, 'startColumnIndex': 19, 'endColumnIndex'

In [47]:
creds = oauthByWeb()
importrangeAllowAccessAutomation("1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM", "1XDBIK9o2k4AVMcW2gfieDZ4qSBxcTW8npw596AY6KCY", creds)

# importrangeAllowAccessAutomation("1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM", "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY", creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=520657704422-o724cfbjdm5vnt6iko73r4buo8gjr68k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fscript.projects+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=I32AlJNeWJ3gztmHdCicizcupti8nu&access_type=offline
JSON 디코딩 실패 )]}'
{"success":true}
